In [1]:
import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *

## s3 bucket name --> yelp_data_parquet , folder--> parquet [store file from kaggle]

## s3 bucket name --> eda_parquet , folder--> parquet  [storing file after doing EDA]


# Reading Business file

In [ ]:
business = spark.read.format("parquet") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .load("s3://yelp_data_parquet/parquet/business.parquet")

In [ ]:
business.cache()

In [ ]:
business1 = business.na.fill(value='Edinburgh',subset=["city"])

In [ ]:
business2 = business1.na.fill(value='',subset=["neighborhood"])

In [ ]:
business3 = business2.na.fill(value='NV',subset=["State"])

In [ ]:
business4 = business3.na.fill(value='',subset=["postal_code"])

In [ ]:
business5 = business4.na.fill(value=43.651070 , subset=["Latitude"])

In [ ]:
business6 = business5.na.fill(value=-79.347015 , subset=["longitude"])

## saving file

# Reading Review

In [ ]:
review_schema = StructType([
    StructField("review_id", StringType()),
    StructField("user_id", StringType()),
    StructField("business_id", StringType()),
    StructField("stars", FloatType()),
    StructField("date", StringType()),
    StructField("text", StringType()),
    StructField("useful", IntegerType()),
    StructField("funny", IntegerType()),
    StructField("cool", IntegerType())
])

In [ ]:
review= spark.read.format("parquet") \
                         .option("header", "true") \
                         .schema(review_schema) \
                         .option("quote", "\"")\
                         .option("escape", "\"")\
                         .option("dateFormat", "yyyy-MM-dd HH:mm:ss")\
                         .option("multiLine",'true')\
                         .load("s3://yelp_data_parquet/parquet/review.parquet")

## saving file